<a href="https://colab.research.google.com/github/Hitomiblood/ProjectsDataScienceMaster/blob/main/CreacionModeloBaseTFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cuaderno de Prueba. Entrenando el modelo:

In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.8/750.8 kB 10.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

Cargamos el DataSet:

In [2]:
%%capture
# Carga de DataSet
!wget https://www.dropbox.com/scl/fi/8bjlyvmulkwb4l09lgbzc/dataSet.zip?rlkey=dmqb2t13gv9ywi4g9bsesdq6q&st=7hkm6ksd&dl=0


In [3]:
!mv dataSet.zip?rlkey=dmqb2t13gv9ywi4g9bsesdq6q dataSet.zip
!unzip dataSet.zip

Archive:  dataSet.zip
  inflating: dataSet.yaml            
   creating: dataSet/images/
   creating: dataSet/images/test/
  inflating: dataSet/images/test/image_0.jpg  
  inflating: dataSet/images/test/image_1.jpg  
  inflating: dataSet/images/test/image_10.jpg  
  inflating: dataSet/images/test/image_100.jpg  
  inflating: dataSet/images/test/image_101.jpg  
  inflating: dataSet/images/test/image_102.jpg  
  inflating: dataSet/images/test/image_103.jpg  
  inflating: dataSet/images/test/image_104.jpg  
  inflating: dataSet/images/test/image_105.jpg  
  inflating: dataSet/images/test/image_106.jpg  
  inflating: dataSet/images/test/image_107.jpg  
  inflating: dataSet/images/test/image_108.jpg  
  inflating: dataSet/images/test/image_109.jpg  
  inflating: dataSet/images/test/image_11.jpg  
  inflating: dataSet/images/test/image_110.jpg  
  inflating: dataSet/images/test/image_111.jpg  
  inflating: dataSet/images/test/image_112.jpg  
  inflating: dataSet/images/test/image_113.jpg  
 

Cargamos un modelo preentrenado:

In [4]:
from ultralytics import YOLO
# Load a pretrained YOLO model (recommended for training)
model = YOLO('yolov8n.pt')

100%|██████████| 6.23M/6.23M [00:00<00:00, 124MB/s]


In [7]:
import os

# Función para corregir las anotaciones
def fix_annotations(annotation_folder):
    for root, dirs, files in os.walk(annotation_folder):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                fix_annotation_file(file_path)

# Función para corregir un archivo de anotación específico
def fix_annotation_file(file_path):
    with open(file_path, "r+") as file:
        lines = file.readlines()
        file.seek(0)
        for line in lines:
            parts = line.strip().split()
            if len(parts) > 5:
                parts = parts[:5]  # Mantener solo las primeras 5 columnas
            fixed_line = " ".join(parts) + "\n"
            file.write(fixed_line)
        file.truncate()

# Directorio donde se encuentran los archivos de anotaciones
annotation_folder = "dataSet/labels/val"

# Corregir las anotaciones
fix_annotations(annotation_folder)

# Directorio donde se encuentran los archivos de anotaciones
annotation_folder = "dataSet/labels/test"

# Corregir las anotaciones
fix_annotations(annotation_folder)
# Directorio donde se encuentran los archivos de anotaciones
annotation_folder = "dataSet/labels/train"

# Corregir las anotaciones
fix_annotations(annotation_folder)


In [8]:
# Train the model using the 'coco8.yaml' dataset for 3 epochs
results = model.train(data='dataSet.yaml', epochs=3)

Ultralytics YOLOv8.2.2 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=dataSet.yaml, epochs=3, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, lin

100%|██████████| 755k/755k [00:00<00:00, 20.8MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/dataSet/labels/train... 1408 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1408/1408 [00:01<00:00, 1294.33it/s]


train: New cache created: /content/dataSet/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

        3/3      3.87G      1.245     0.7734     0.9378        592        640: 100%|██████████| 88/88 [00:11<00:00,  7.92it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01<00:00,  5.67it/s]

                   all        228       8684       0.85       0.87      0.931      0.569



3 epochs completed in 0.017 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.2 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:26<00:00,  3.37s/it]


                   all        228       8684      0.827      0.884      0.924      0.587
Speed: 0.1ms preprocess, 0.5ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/train


In [9]:
metrics = model.val()

Ultralytics YOLOv8.2.2 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/dataSet/labels/val.cache... 228 images, 0 backgrounds, 0 corrupt: 100%|██████████| 228/228 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:28<00:00,  1.93s/it]


                   all        228       8684      0.827      0.884      0.924      0.588
Speed: 0.1ms preprocess, 2.6ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect/train2


In [11]:
metrics.box.maps

array([    0.58783])

In [ ]:
model.predict(source = 'dataSet/images/test/image_0.jpg', show = True, save = True)